# P1: Modelado de un vehículo

# P2: Segimiento de Caminos

# P3: SLAM basado en EFK

# P4: Evitar Obstáculos

# P5: Planificación de Caminos - Dijkstra

P6: Planificación de Caminos - A*

P7: Navegación Autónoma


<div style="color: red;">

- Pendiente mejora del código aestrella (p. ej., while true/break)
- Pendiente corregir bug de A* cuando la heurística está bien/mal
    
    
</div>

\newpage

# Planificación de Caminos - A*

En este ejercicio se plantea realizar una planificación de caminos para un robot móvil, siendo necesario planificar el camino a realizar entre dos puntos o localizaciones muy distantes en el espacio. Para resolver este problema de planificación, dado que la ubicación inicial y final están separadas por largas distancias, se utilizan mapas topológicos y algoritmos de búsqueda en grafos. 

En esta práctica se plantea resolver el problema de encontrar un camino o ruta mediante el algoritmo A* (A estrella). Este algoritmo es un método de búsqueda informada en grafos, que considera el coste de los caminos que unen los nodos del grafo y una heurística que da una estimación del coste de llegar al objetivo por el camino explorado. El algoritmo A* proporciona la ruta de menor coste entre un origen y un destino, siendo, por tanto, la ruta óptima y, además, encuentra la solución con el menor coste computacional. Ambas afirmaciones son ciertas siempre que la heurística sea consistente: 

$$f(n) = g(n) + h^*(n) , \quad h^*(n) \le h(n)$$

\newpage

## Ejercicio

Implementar el algoritmo A* en Matlab mediante una función que devuelva el coste y la ruta óptima a partir de un origen y un destino pasados como parámetros, además del mapa topológico o grafo, que se le pasará a la función como una matriz NxN, que almacena el coste de llegar del nodo n1, como fila, al nodo n2, como columna, y, por último, la heurística. La función se debe implementar de forma que la llamada: 
```
>> [coste, ruta]=aestrella(G,H,1,7) 

```

devuelva el coste de llegar desde el nodo origen al nodo destino, y un vector con la lista de nodos que componen la ruta (incluidos los nodos inicial y final). 


Por ejemplo, dado el mapa topológico y la matriz de costes correspondiente que se muestran a continuación: 

![E61enunciadoA.jpg](E61enunciadoA.jpg)

La heurística será otra matriz NxN que muestra, para cada nodo, el coste estimado para llegar a un nodo concreto. 

![E61enunciadoA.jpg](E61enunciadoB.jpg)

### Codificación

```C
function [coste, ruta] = aestrella(G, H, Origen, Destino)
    % G: matriz de adyacencia con los costes reales entre nodos
    % H: matriz heurística: coste estimado desde cada nodo hasta cualquier otro
    % Origen: nodo de inicio
    % Destino: nodo objetivo
    % Salida:
    %   - coste: coste total del camino más corto
    %   - ruta: lista de nodos que forman el camino más corto (incluye Origen y Destino)

    n = size(G, 1);             % Número total de nodos en el grafo
    g = Inf(n, 1);              % g(n): coste real mínimo desde el Origen a cada nodo
    f = Inf(n, 1);              % f(n) = g(n) + h(n): estimación total de coste
    g(Origen) = 0;              % El coste real desde el Origen a sí mismo es 0
    f(Origen) = H(Origen, Destino);  % f del Origen = heurística inicial

    nodo_ant = NaN(n, 1);       % Para reconstruir el camino más corto
    visitados = false(n, 1);    % Marcador de nodos ya procesados

    while true
        % Buscar el nodo no visitado con menor f(n)
        min_f = Inf;
        u = -1;  % nodo actual
        for i = 1:n
            if ~visitados(i) && f(i) < min_f
                min_f = f(i);
                u = i;
            end
        end

        if u == -1
            break;  % No quedan nodos por explorar: no hay camino posible
        end

        if u == Destino
            break;  % Hemos llegado al destino
        end

        visitados(u) = true;  % Marcar el nodo actual como procesado

        % Recorrer todos los vecinos del nodo actual u
        for v = 1:n
            if G(u,v) > 0 && ~visitados(v)
                % --------------------------------------------------------
                % DEFINICIÓN DEL COSTE TENTATIVO:
                % Es el coste de ir desde el nodo Origen hasta v pasando por u.
                % Es decir, g(u) + coste de la arista de u a v.
                % Se compara con el g(v) actual para ver si se mejora.
                % --------------------------------------------------------
                tentativo_g = g(u) + G(u,v);

                % Si es mejor que el coste conocido hasta ahora, lo actualizamos
                if tentativo_g < g(v)
                    g(v) = tentativo_g;
                    f(v) = g(v) + H(v, Destino);  % f(n) = g(n) + h(n)
                    nodo_ant(v) = u;              % Guardar nodo anterior
                end
            end
        end
    end

    % Reconstrucción del camino desde el destino hacia atrás
    ruta = Destino;
    u = Destino;
    while ~isnan(nodo_ant(u))
        u = nodo_ant(u);
        ruta = [u ruta];  % Prependemos cada nodo al camino
    end

    % Si no comienza por el origen, significa que no hay camino posible
    if ruta(1) ~= Origen
        ruta = [];
        coste = NaN;
    else
        coste = g(Destino);  % El coste total es el g del nodo destino
    end
end
```

### Pruebas

```C
>> load('../P5/grafos.mat')
>> [coste, ruta] = aestrella(G, H, 1, 7)
coste =
     7
ruta =
     1     2     4     6     7
```

\newpage

## Ejercicio

Comprobar el resultado del algoritmo para los siguientes nodos inicial y final:  

```
>> [coste, ruta]=aestrella(G,H,7,4) 
```

Obsérvese como el resultado no es óptimo debido a que, para el camino óptimo, la heurística ha sobreestimado el coste real, por tanto, la heurística no es admisible, ya que no se cumple la desigualdad h*(n) ≤ h(n). 

### Resultados

La heurística propuesta no cumple con que el coste tiene que ser una estimación optimista de la distancia entre los nodos, y por tanto el resultado de la ejecución no es adecuado ya que el camino óptimo sería [7, 6, 4] con un coste de 1+2=3:

```C
>> [coste, ruta] = aestrella(G, H, 7, 4)
coste =
     4
ruta =
     7     4
```


\newpage

## Ejercicio

Proponga una heurística admisible para que el resultado anterior si se corresponda al camino óptimo entre ambos nodos. 

### Modifiación de la heurśitcia

El problema en este caso es que la matriz H está mal definida porque la distancia estimada entre el nodo 6 y el nodo 4 (fila 6, columna 4) es 4, superior a la distancia real en el grafo que es 2:

```
H = [
     0     2     5     5     5     6     9
     2     0     2     3     6     6     8
     5     2     0     2     6     6     7
     5     3     2     0     5     4     6
     5     6     6     5     0     1     5
     6     6     6     4     1     0     1
     9     8     7     6     5     1     0]
```

Para corregir el problema, basta con reducir su valor a 2 o menos, por ejemplo: 

```C
>> H(6,4) = 1;
>> [coste, ruta] = aestrella(G, H, 7, 4)
coste =
     3
ruta =
     7     6     4
```

Otra posible solución sería convertir utilizar una matriz de unos:

```C
>> ones(7);
>> [coste, ruta] = aestrella(G, H, 7, 4)
coste =
     3
ruta =
     7     6     4
```

\newpage

## Apéndice: Solución en Python

### Función auxiliar para obtener trayectoria

In [1]:
def trayectoria(costes, inicio, fin):
    if inicio == fin:
        return([costes[fin]])
    else:
        nomPrev = costes[fin][3]
        idxPrev = next((i for i, x in enumerate(costes) if x[1] == nomPrev), None)
        nodPrev = costes[idxPrev]
        return trayectoria(costes, inicio, nodPrev[0])+[costes[fin]]

### Algoritmo A*

In [2]:
def aestrella(Adjacencia, Heuristica, nombres, inicio, fin):
    nodos =  [x for x in range(len(Adjacencia))]
    infinito = 1000
    ahora = inicio
    visitados = []
    
    # inicializa la lista de costes, cada uno con (id, nombre, coste, h)
    #costes = [[n, nombres[n], 0 if n==inicio else infinito, None, h[n]] for n in nodos]
    costes = [[n, nombres[n], infinito, None, infinito] for n in nodos]
    costes[inicio] = [inicio, nombres[inicio], 0, None, 0]
    while ahora != fin:
        
        # añadir el nodo actual a la lista de visitados
        visitados.append(ahora)
        #print("-------\n", costes[ahora])
        
        # actualizar los costes de todos los nodos adjacentes
        coste = costes[ahora][2]
        #print("ahora={}, coste={}, h={}".format(nombres[ahora], coste, h))
        for i,a in enumerate(Adjacencia[ahora]):
            if i not in visitados and a > 0 and coste+a < costes[i][2]: 
                #print("i={}, a={}, h={}".format(nombres[i], a, h[i]))            
                h = Heuristica[i][ahora]    
                #costes[i] = [i, nombres[i], coste+a, nombres[ahora], coste+a+h[i]]
                costes[i] = [i, nombres[i], coste+a, nombres[ahora], coste+a+h]
        
        # reordenar la lista de pendientes y quedarse con el de menor coste
        pendientes = sorted([x for x in costes if x[0] not in visitados], key=lambda x: x[4])
        #print(pendientes)
        ahora = pendientes[0][0]
    if costes[fin][2] < infinito:
        ruta = trayectoria(costes, inicio, fin)
        return costes[fin][2],[x[1] for x in ruta]        
    else:
        return ['Infinito', 'Nodo inalcanzable']
    
        

### Pruebas con grafo de los apuntes

Definimos una red de adayacencia que se corresponde con el grafo de los apuntes. Los nombres de los nodos se asignan en la variable Nombres, y la matriz heurística sólo se define para la última fila

In [3]:
AdjacenciaTeo = [
    [0, 7, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0], # S
    [7, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0], # A
    [2, 3, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0], # B
    [0, 4, 4, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0], # C
    [0, 0, 1, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0], # D
    [0, 0, 0, 5, 3, 0, 0, 0, 0, 0, 0, 0, 0], # E
    [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2], # F
    [2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0], # G
    [0, 0, 0, 0, 0, 0, 0, 2, 0, 4, 4, 0, 0], # H
    [0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 6, 4, 0], # I
    [0, 0, 0, 0, 0, 0, 0, 0, 4, 6, 0, 4, 0], # J
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 5], # K
    [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 5, 0] # X
]
Nombres = "SABCDEFGHIJKX"
Heuristica = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # S
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # A
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # B
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # C
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # D
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # E
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # F
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # G
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # H
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # I
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # J
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # K
    [6, 6, 5, 4, 3, 3, 2, 7, 6, 4, 6, 4, 0] # X
]

In [4]:
inicio, fin = 0, 12
total, ruta = aestrella(AdjacenciaTeo, Heuristica, Nombres, inicio, fin)
print("A*:      total desde {} hasta {} = {}, ruta = {}".format(
    Nombres[inicio], Nombres[fin], total, ruta))
# errata en pagina 68, en C es 6|10(B)

A*:      total desde S hasta X = 7, ruta = ['S', 'B', 'D', 'F', 'X']


### Pruebas con la red de 7 nodos

In [5]:
AdjacenciaP5a = [ # matriz de costes y adyacencia
    [0, 2, 0, 0, 9, 0, 0],
    [2, 0, 1, 2, 0, 0, 0],
    [0, 1, 0, 5, 0, 0, 0],
    [0, 2, 5, 0, 1, 2, 4],
    [9, 0, 0, 1, 0, 4, 0],
    [0, 0, 0, 2, 4, 0, 1],
    [0, 0, 0, 4, 0, 1, 0]]
Heuristica = [
    [0, 2, 5, 5, 5, 6, 9],
    [2, 0, 2, 3, 6, 6, 8],
    [5, 2, 0, 2, 6, 6, 7],
    [5, 3, 2, 0, 5, 4, 6], 
    [5, 6, 6, 5, 0, 1, 5],
    [6, 6, 6, 4, 1, 0, 1],
    [9, 8, 7, 6, 5, 1, 0]]
Nombres = "1234567"
inicio, fin = 0, 6
total, ruta = aestrella(AdjacenciaP5a, Heuristica, Nombres, inicio, fin)
print("A*:      total desde {} hasta {} = {}, ruta = {}".format(
    Nombres[inicio], Nombres[fin], total, ruta))

A*:      total desde 1 hasta 7 = 7, ruta = ['1', '2', '4', '6', '7']


In [6]:
inicio, fin = 6, 3
total, ruta = aestrella(AdjacenciaP5a, Heuristica, Nombres, inicio, fin)
print("A*:      total desde {} hasta {} = {}, ruta = {}".format(
    Nombres[inicio], Nombres[fin], total, ruta))

A*:      total desde 7 hasta 4 = 3, ruta = ['7', '6', '4']


### Corrección de la heurística


Cambiamos la fila 4 para modificar la heurística cuando el destino es ese nodo:

- El nodo 4 sigue a cero porque es el nodo destino
- A los nodos directamente conectados al 4 (todos menos el 1) se les asignan los valores de la matriz de adyacencica
- Al nodo 1 se le asigna un valor 2 (al no estar directamente conectado al menos hay dos saltos)

In [7]:
Heuristica[5][3] = 1
Heuristica

[[0, 2, 5, 5, 5, 6, 9],
 [2, 0, 2, 3, 6, 6, 8],
 [5, 2, 0, 2, 6, 6, 7],
 [5, 3, 2, 0, 5, 4, 6],
 [5, 6, 6, 5, 0, 1, 5],
 [6, 6, 6, 1, 1, 0, 1],
 [9, 8, 7, 6, 5, 1, 0]]

In [8]:
total, ruta = aestrella(AdjacenciaP5a, Heuristica, Nombres, inicio, fin)
print("A*:      total desde {} hasta {} = {}, ruta = {}".format(
    Nombres[inicio], Nombres[fin], total, ruta))

A*:      total desde 7 hasta 4 = 3, ruta = ['7', '6', '4']


\newpage

## Apéndice: Implementación con Orientación a Objetos

### Clase nodo

In [9]:
class nodo:
    def __init__(self, idx, nombre):
        self.idx = idx
        self.nombre = nombre
        self.desde = None # coste desde inicio al nodo
        self.hasta = None # coste desde nodo hasta fin
        self.previo = None # nodo previo en el camino
    def AsignarDesde(self, nuevocoste, previo):
        if self.desde == None or self.desde > nuevocoste:
            self.desde = nuevocoste
            self.previo = previo
    def AsignarHasta(self, nuevocoste):
        if self.hasta == None or self.hasta > nuevocoste:
            self.hasta = nuevocoste
    def __str__(self):
        return("idx={}, nombre={}, desde={}, hasta={}, previo={}".format(
            self.idx, self.nombre, self.desde, self.hasta, self.previo))
#n = nodo(0, 'A')        

### Cálculo de trayectoria

In [10]:
def trayectoria2(nodos, inicio, fin):
    #print("trayectoria2, inicio={}, fin={}".format(inicio, fin))
    if inicio == fin:
        return([nodos[fin].nombre])
    else:
        nomPrev = nodos[fin].previo
        idxPrev = next((i for i, n in enumerate(nodos) if n.nombre == nomPrev), None)
        return trayectoria2(nodos, inicio, idxPrev)+[nodos[fin].nombre]

### Clase para almacenar un grafo

In [11]:
import numpy as np

class grafo:
    def __init__(self, Nombres, Adyacencia, Heuristica=None):
        #print("h ", Heuristica)
        self.Nombres = Nombres
        self.Adyacencia = np.array(Adyacencia)
        self.N = len(Adyacencia)
        if Heuristica == None:
            # asignar el valor 2 a los nodos no conectados directamente
            self.Heuristica = 2*np.ones((self.N, self.N))
            for i in range(self.N):
                for j in range(self.N):
                    if Adyacencia[i][j] != 0 or i == j: 
                        #print("i", i, ", j", j, ", adj", Adyacencia[i][j])
                        self.Heuristica[i][j] = Adyacencia[i][j]
        else:
            #self.Heuristica = copy.deepcopy(Heuristica)
            self.Heuristica = np.array(Heuristica)
    def camino(self, inicio, fin, bAestrella = False):
        # inicialización        
        nodos =  [nodo(i, Nombres[i]) for i in range(self.N)]
        ahora = inicio
        nodos[inicio].desde = 0
        visitados = []
        h = self.Heuristica[fin]
        if bAestrella:
            ordenar = lambda x: 100000 if x.hasta==None else x.hasta
        else:
            ordenar = lambda x: 100000 if x.desde==None else x.desde
        while ahora != fin:
            # añadir el nodo actual a la lista de visitados
            visitados.append(ahora)
            
            # actualizar los costes de todos los nodos adjacentes
            #print(self.Adyacencia[ahora])
            for i, arco in enumerate(self.Adyacencia[ahora]):
                if i not in visitados and arco > 0:
                    print("i={}, arco={}, previo={}".format(i, arco, nodos[ahora].nombre))
                    nodos[i].AsignarDesde(nodos[ahora].desde + arco, nodos[ahora].nombre)
                    nodos[i].AsignarHasta(nodos[i].desde + h[i])

            # reordenar la lista de pendientes y quedarse con el de menor coste
            pendientes = sorted([x for x in nodos if x.idx not in visitados], key=ordenar)
            ahora = pendientes[0].idx
        #for i in range(self.N):
        #    print(nodos[i])
        #return
        if nodos[fin].desde == None:
            return ['Infinito', 'Nodo inalcanzable']
        else:
            #return nodos
            ruta = trayectoria2(nodos, inicio, fin)
            #print(ruta)
            return int(nodos[fin].hasta) ,ruta # [x[1] for x in ruta]        

                


In [12]:
Nombres = "SABCDEFGHIJKX"
Heuristica = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # S
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # A
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # B
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # C
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # D
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # E
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # F
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # G
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # H
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # I
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # J
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # K
    [6, 6, 5, 4, 3, 3, 2, 7, 6, 4, 6, 4, 0] # X
]
G = grafo(Nombres, AdjacenciaTeo, Heuristica)

In [13]:
# Ejecución con la opción A*
inicio, fin, bAestrella = 0, 12, True
G.camino(inicio, fin, bAestrella)


i=1, arco=7, previo=S
i=2, arco=2, previo=S
i=7, arco=2, previo=S
i=1, arco=3, previo=B
i=3, arco=4, previo=B
i=4, arco=1, previo=B
i=5, arco=3, previo=D
i=6, arco=2, previo=D
i=12, arco=2, previo=F


(7, ['S', 'B', 'D', 'F', 'X'])

In [15]:
# Ejecución con la opción Djistra (bAestrella=False)
inicio, fin, bAestrella = 0, 12, False
G.camino(inicio, fin, bAestrella)

i=1, arco=7, previo=S
i=2, arco=2, previo=S
i=7, arco=2, previo=S
i=1, arco=3, previo=B
i=3, arco=4, previo=B
i=4, arco=1, previo=B
i=8, arco=2, previo=G
i=5, arco=3, previo=D
i=6, arco=2, previo=D
i=9, arco=4, previo=H
i=10, arco=4, previo=H
i=3, arco=4, previo=A
i=12, arco=2, previo=F


(7, ['S', 'B', 'D', 'F', 'X'])

In [16]:
AdjacenciaP5b=[
[0, 2, 0, 0, 0, 2, 3, 0,  0, 0, 0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[2, 0, 2, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 2, 0, 2, 0, 0, 0, 2,  0, 0, 0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 2, 0, 0, 0, 0, 3,  0, 0, 0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  3, 2, 0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[2, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[3, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 3,  0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 2, 3, 0, 0, 0, 0,  2, 0, 0, 0, 13, 0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 3, 0, 0, 2,  0, 0, 0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 2, 0, 0, 0,  0, 0, 0, 0, 0,  0, 2, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 2, 0,  0, 0, 2,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 2, 0, 0,  0, 0, 0,  2, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 3, 13, 0, 0, 0, 0, 0,  0, 0, 0,  3, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 2, 0,  0, 3, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 2, 0, 0, 0,  2, 0, 0,  0, 0, 0,  2, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 2, 0, 0,  0, 0, 0,  8, 0, 0,  0, 6, 3,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 2, 3,  0, 0, 8,  0, 0, 0,  0, 0, 10, 3, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  3, 0, 0,  0, 0, 2,  0, 0, 0,  2, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0,  0, 2, 0,  2, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 2, 0,  0, 0, 10, 0, 0, 0,  0, 0, 2],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 6,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 3, 10, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0,  0, 2, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0,  0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0,  0, 0, 0,  2, 0, 0,  0, 0, 0]]
Nombres=list(range(1, 26))
G = grafo(Nombres, AdjacenciaP5b) # construye su propia heurística

In [17]:
# Ejecución con la opción Djistra (bAestrella=False)
inicio, fin, bAestrella = 22, 21, False
G.camino(inicio, fin, bAestrella)

i=17, arco=2, previo=23
i=13, arco=3, previo=18
i=18, arco=2, previo=18
i=19, arco=2, previo=19
i=14, arco=2, previo=14
i=14, arco=2, previo=20
i=24, arco=2, previo=20
i=9, arco=2, previo=15
i=4, arco=2, previo=10
i=8, arco=3, previo=5
i=7, arco=2, previo=9
i=2, arco=2, previo=8
i=3, arco=3, previo=8
i=12, arco=13, previo=8
i=1, arco=2, previo=3
i=3, arco=2, previo=3
i=0, arco=2, previo=2
i=5, arco=2, previo=1
i=6, arco=3, previo=1
i=12, arco=3, previo=7
i=16, arco=3, previo=13
i=11, arco=2, previo=17
i=15, arco=8, previo=17
i=21, arco=10, previo=17
i=10, arco=2, previo=12
i=15, arco=2, previo=11
i=20, arco=6, previo=16
i=21, arco=3, previo=16


(40, [23, 18, 14, 15, 10, 5, 9, 8, 3, 2, 1, 7, 13, 17, 12, 11, 16, 22])

In [18]:
# Ejecución con la opción Djistra (bAestrella=False)
inicio, fin, bAestrella = 22, 21, True
G.camino(inicio, fin, bAestrella)

i=17, arco=2, previo=23
i=13, arco=3, previo=18
i=18, arco=2, previo=18
i=19, arco=2, previo=19
i=14, arco=2, previo=14
i=14, arco=2, previo=20
i=24, arco=2, previo=20
i=9, arco=2, previo=15
i=4, arco=2, previo=10
i=8, arco=3, previo=5
i=7, arco=2, previo=9
i=2, arco=2, previo=8
i=3, arco=3, previo=8
i=12, arco=13, previo=8
i=1, arco=2, previo=3
i=3, arco=2, previo=3
i=0, arco=2, previo=2
i=5, arco=2, previo=1
i=6, arco=3, previo=1
i=12, arco=3, previo=7
i=16, arco=3, previo=13
i=11, arco=2, previo=17
i=15, arco=8, previo=17
i=21, arco=10, previo=17
i=10, arco=2, previo=12
i=15, arco=2, previo=11
i=20, arco=6, previo=16
i=21, arco=3, previo=16


(40, [23, 18, 14, 15, 10, 5, 9, 8, 3, 2, 1, 7, 13, 17, 12, 11, 16, 22])